In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"daily": ["sunset", "sunrise", "wind_speed_10m_max", "wind_gusts_10m_max", "uv_index_max", "weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "daylight_duration", "sunshine_duration", "uv_index_clear_sky_max", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_sum", "precipitation_hours", "precipitation_probability_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
	"timezone": "Europe/London"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_sunset = daily.Variables(0).ValuesInt64AsNumpy()
daily_sunrise = daily.Variables(1).ValuesInt64AsNumpy()
daily_wind_speed_10m_max = daily.Variables(2).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(3).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(4).ValuesAsNumpy()
daily_weather_code = daily.Variables(5).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(6).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(7).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(8).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(9).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(10).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(11).ValuesAsNumpy()
daily_uv_index_clear_sky_max = daily.Variables(12).ValuesAsNumpy()
daily_rain_sum = daily.Variables(13).ValuesAsNumpy()
daily_showers_sum = daily.Variables(14).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(15).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(16).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(17).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(18).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(19).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(20).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(21).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["sunset"] = daily_sunset
daily_data["sunrise"] = daily_sunrise
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["uv_index_clear_sky_max"] = daily_uv_index_clear_sky_max
daily_data["rain_sum"] = daily_rain_sum
daily_data["showers_sum"] = daily_showers_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone b'Europe/London'b'GMT+1'
Timezone difference to GMT+0 3600 s
                       date      sunset     sunrise  wind_speed_10m_max  \
0 2025-07-17 23:00:00+00:00  1752866341  1752807968            8.669949   
1 2025-07-18 23:00:00+00:00  1752952668  1752894450            8.905908   
2 2025-07-19 23:00:00+00:00  1753038993  1752980933           10.948973   
3 2025-07-20 23:00:00+00:00  1753125316  1753067417           14.113653   
4 2025-07-21 23:00:00+00:00  1753211636  1753153901           17.533146   
5 2025-07-22 23:00:00+00:00  1753297954  1753240387           18.391735   
6 2025-07-23 23:00:00+00:00  1753384271  1753326873           10.495713   

   wind_gusts_10m_max  uv_index_max  weather_code  temperature_2m_max  \
0           20.160000          5.30           3.0           24.195499   
1           19.440001          6.65          45.0           27.595499   
2           31.319998          5.65 

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Step 1: Dummy task schedule
tasks = pd.DataFrame({
    "task_id": ["A", "B", "C"],
    "task_name": ["Excavation", "Foundation Pour", "Concrete Curing"],
    "duration": [3, 4, 2],
    "depends_on": [[], ["A"], ["B"]],
    "task_type": ["Excavation", "Concrete", "Concrete"]
})

# Step 2: Simulated 15-day weather forecast
start_date = datetime.strptime("2025-07-01", "%Y-%m-%d")
weather_data = pd.DataFrame({
    "date": [start_date + timedelta(days=i) for i in range(15)],
    "weather": np.random.choice(["Clear", "Rain", "Thunderstorm"], size=15, p=[0.6, 0.3, 0.1]),
    "temp": np.random.randint(20, 35, size=15),
    "humidity": np.random.randint(40, 100, size=15),
    "wind": np.random.randint(5, 20, size=15)
})

# Step 3: Dummy ML model training with sample historical data
# Features: task_type (encoded), weather, temp, humidity, wind
historical_data = pd.DataFrame({
    "task_type": ["Excavation", "Excavation", "Concrete", "Concrete"],
    "weather": ["Clear", "Rain", "Clear", "Thunderstorm"],
    "temp": [28, 22, 30, 25],
    "humidity": [50, 80, 45, 90],
    "wind": [10, 12, 8, 18],
    "delay": [0, 2, 0, 3]
})

# Encoding categorical variables
le_task = LabelEncoder()
le_weather = LabelEncoder()
historical_data["task_type_enc"] = le_task.fit_transform(historical_data["task_type"])
historical_data["weather_enc"] = le_weather.fit_transform(historical_data["weather"])

# Train ML model
X_train = historical_data[["task_type_enc", "weather_enc", "temp", "humidity", "wind"]]
y_train = historical_data["delay"]
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Step 4: Apply model to forecast
# Schedule and predict delay for each task
schedule = []
task_start_dates = {}

for idx, row in tasks.iterrows():
    task_type_enc = le_task.transform([row["task_type"]])[0]
    dependencies = row["depends_on"]
    earliest_start = start_date if not dependencies else max(task_start_dates[dep]["end"] for dep in dependencies)

    # Check weather on planned task days
    task_days = [earliest_start + timedelta(days=i) for i in range(row["duration"])]
    delays = 0
    for day in task_days:
        if day in list(weather_data["date"]):
            w = weather_data[weather_data["date"] == day].iloc[0]
            weather_enc = le_weather.transform([w["weather"]])[0]
            features = [[task_type_enc, weather_enc, w["temp"], w["humidity"], w["wind"]]]
            predicted_delay = int(round(model.predict(features)[0]))
            delays = max(delays, predicted_delay)  # use max delay over all task days

    actual_start = earliest_start + timedelta(days=delays)
    actual_end = actual_start + timedelta(days=row["duration"] - 1)

    task_start_dates[row["task_id"]] = {"start": actual_start, "end": actual_end, "delay": delays}
    schedule.append({
        "task_id": row["task_id"],
        "task_name": row["task_name"],
        "start": actual_start.strftime("%Y-%m-%d"),
        "end": actual_end.strftime("%Y-%m-%d"),
        "predicted_delay": delays
    })

schedule_df = pd.DataFrame(schedule)
schedule_df


/home/chandan/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/chandan/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/chandan/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/chandan/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/chandan/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted w

,task_id,task_name,start,end,predicted_delay
0,A,Excavation,2025-07-03,2025-07-05,2
1,B,Foundation Pour,2025-07-06,2025-07-09,1
2,C,Concrete Curing,2025-07-10,2025-07-11,1


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Define tasks data
tasks_data = {
    'task_id': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9'],
    'task_name': ['Excavation', 'Soil Compaction', 'Foundation Pour', 'Basement Waterproofing', 
                  'Backfilling', 'Slab Casting', 'Ground Floor Column', 'Ground Floor Walls', 'Roof Slab'],
    'duration_days': [7, 6, 11, 5, 4, 7, 4, 5, 9],
    'depends_on': ['', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8'],
    'weather_sensitive': [1, 1, 1, 1, 1, 1, 0, 0, 0],
    'start_date': ['2025-05-01', '2025-05-08', '2025-05-14', '2025-05-25', '2025-05-30', 
                   '2025-06-03', '2025-06-10', '2025-06-14', '2025-06-19'],
    'end_date': ['2025-05-07', '2025-05-13', '2025-05-24', '2025-05-29', '2025-06-02', 
                 '2025-06-09', '2025-06-13', '2025-06-18', '2025-06-27']
}
tasks_df = pd.DataFrame(tasks_data)
tasks_df['start_date'] = pd.to_datetime(tasks_df['start_date'])
tasks_df['end_date'] = pd.to_datetime(tasks_df['end_date'])

# Define weather data (subset relevant to task periods)
weather_data = {
    'date': ['2025-05-01', '2025-05-02', '2025-05-03', '2025-05-04', '2025-05-05', '2025-05-06', '2025-05-07', 
             '2025-05-08', '2025-05-09', '2025-05-10', '2025-05-11', '2025-05-12', '2025-05-13', 
             '2025-05-14', '2025-05-15', '2025-05-16', '2025-05-17', '2025-05-18', '2025-05-19', 
             '2025-05-20', '2025-05-21', '2025-05-22', '2025-05-23', '2025-05-24', 
             '2025-05-25', '2025-05-26', '2025-05-27', '2025-05-28', '2025-05-29', 
             '2025-05-30', '2025-05-31', '2025-06-01', '2025-06-02', 
             '2025-06-03', '2025-06-04', '2025-06-05', '2025-06-06', '2025-06-07', '2025-06-08', '2025-06-09', 
             '2025-06-10', '2025-06-11', '2025-06-12', '2025-06-13', '2025-06-14', '2025-06-15', 
             '2025-06-16', '2025-06-17', '2025-06-18', '2025-06-19', '2025-06-20', '2025-06-21', 
             '2025-06-22', '2025-06-23', '2025-06-24', '2025-06-25', '2025-06-26', '2025-06-27'],
    'precipitation_sum': [0, 0.6, 1.1, 0.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.2, 0.7, 0.3, 
                         0, 0, 0.2, 0.7, 0, 2.1, 0.3, 0.8, 5.6, 0.2, 0.3, 0, 0.7, 0, 
                         0.3, 0.2, 0.2, 2.3, 4.4, 0, 0, 0.2, 0, 0, 0, 0, 0.3, 
                         0, 0, 0, 0, 0, 0, 0, 1.5, 0.5, 3.5, 3.9, 1.7]
}
weather_df = pd.DataFrame(weather_data)
weather_df['date'] = pd.to_datetime(weather_df['date'])

# Function to predict weather delays
def predict_weather_delays(tasks_df, weather_df):
    tasks_df['weather_delay_days'] = 0
    for index, task in tasks_df.iterrows():
        if task['weather_sensitive'] == 1:
            task_weather = weather_df[(weather_df['date'] >= task['start_date']) & 
                                     (weather_df['date'] <= task['end_date'])]
            delay_days = (task_weather['precipitation_sum'] > 2).sum()
            tasks_df.at[index, 'weather_delay_days'] = delay_days
    tasks_df['new_duration'] = tasks_df['duration_days'] + tasks_df['weather_delay_days']
    return tasks_df

# Function to recalculate schedule
def recalculate_schedule(tasks_df):
    tasks_df['new_start_date'] = pd.NaT
    tasks_df['new_end_date'] = pd.NaT
    for index, task in tasks_df.iterrows():
        if not task['depends_on']:
            tasks_df.at[index, 'new_start_date'] = task['start_date']
        else:
            predecessor_end = tasks_df[tasks_df['task_id'] == task['depends_on']]['new_end_date'].iloc[0]
            tasks_df.at[index, 'new_start_date'] = predecessor_end + timedelta(days=1)
        tasks_df.at[index, 'new_end_date'] = tasks_df.at[index, 'new_start_date'] + \
                                            timedelta(days=task['new_duration'] - 1)
    return tasks_df

# Apply weather delays
tasks_df = predict_weather_delays(tasks_df, weather_df)

# Recalculate schedule
tasks_df = recalculate_schedule(tasks_df)

# Format dates for output
tasks_df['new_start_date'] = tasks_df['new_start_date'].dt.strftime('%Y-%m-%d')
tasks_df['new_end_date'] = tasks_df['new_end_date'].dt.strftime('%Y-%m-%d')

# Print updated schedule
print("Updated Project Schedule with Weather Delays:")
print(tasks_df[['task_id', 'task_name', 'new_start_date', 'new_end_date', 'new_duration']])

# Calculate total project delay
original_end = tasks_df['end_date'].max()
new_end = pd.to_datetime(tasks_df['new_end_date'].max())
total_delay = (new_end - original_end).days
print(f"\nTotal Project Delay due to Weather: {total_delay} days")

Updated Project Schedule with Weather Delays:
  task_id               task_name new_start_date new_end_date  new_duration
0      T1              Excavation     2025-05-01   2025-05-07             7
1      T2         Soil Compaction     2025-05-08   2025-05-13             6
2      T3         Foundation Pour     2025-05-14   2025-05-24            11
3      T4  Basement Waterproofing     2025-05-25   2025-05-31             7
4      T5             Backfilling     2025-06-01   2025-06-04             4
5      T6            Slab Casting     2025-06-05   2025-06-13             9
6      T7     Ground Floor Column     2025-06-14   2025-06-17             4
7      T8      Ground Floor Walls     2025-06-18   2025-06-22             5
8      T9               Roof Slab     2025-06-23   2025-07-01             9

Total Project Delay due to Weather: 4 days
